# Title

In [ ]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'  # always print last expr.
%config InlineBackend.figure_format = 'svg'
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.set_printoptions(precision=4, floatmode="fixed", suppress=True)
rng = np.random.default_rng()

In [ ]:
def compute(s):
    print(f"Computing {s}...")
    return f"{s} result"


class MethodType:
    "Emulate PyMethod_Type in Objects/classobject.c"

    def __init__(self, func, obj):
        self.__func__ = func
        self.__self__ = obj

    def __call__(self, *args, **kwargs):
        func = self.__func__
        obj = self.__self__
        return func(obj, *args, **kwargs)


def issubclassable(cls):
    try:

        class _(cls):
            ...

    except:
        return False
    return True


class BaseDecorator:
    def __init__(self, obj):
        if hasattr(obj, "__func__"):
            self.__func__ = obj.__func__
        else:  # was never decorated before.
            self.__func__ = obj

    def __call__(self, *args, **kwargs):
        return self.__func__.__call__(*args**kwargs)


def Property(func):
    class Wrapped(property):
        @property
        def __func__(self):
            return self.fget

    return Wrapped(func)


def ClassMethod(func):
    if issubclassable(type(func)):

        class Wrapped(type(func)):
            def __get__(self, obj, cls=None):
                if cls is None:
                    cls = type(obj)
                if hasattr(type(self.__func__), "__get__"):
                    return self.__func__.__get__(cls)
                return MethodType(self.__func__, cls)

        return Wrapped(func)

    return classmethod(func)

In [ ]:
class A:
    # python 3.9

    @classmethod
    @property
    def clsproperty(cls):
        """A python 3.9 class-property"""
        return compute(f"{cls.__name__}'s clsproperty")

    @classmethod
    def clsmethod(cls):
        """A python 3.9 class-method"""
        return compute(f"{cls.__name__}'s clsmethod")

    @property
    def instproperty(self):
        """A python 3.9 instance-property"""
        return compute(f"{self}'s instproperty")

    # our modified versions

    @ClassMethod
    @Property
    def myclsproperty(cls):
        """A custom class-property"""
        return compute(f"{cls.__name__}'s myclsproperty")

    @ClassMethod
    def myclsmethod(cls):
        """A custom classmethod"""
        return compute(f"{cls.__name__}'s myclsmethod")

    @Property
    def myinstproperty(self):
        """A custom instance-property"""
        return compute(f"{self}'s myinstproperty")


print(A.__dict__)
help(A)

In [ ]:
class MethodType:
    "Emulate PyMethod_Type in Objects/classobject.c"

    def __init__(self, func, obj):
        self.__func__ = func
        self.__self__ = obj

    def __call__(self, *args, **kwargs):
        func = self.__func__
        obj = self.__self__
        return func(obj, *args, **kwargs)


class ClassMethod:
    "Emulate PyClassMethod_Type() in Objects/funcobject.c"

    def __init__(self, f):
        self.f = f

    def __get__(self, obj, cls=None):
        if cls is None:
            cls = type(obj)
        if hasattr(type(self.f), "__get__"):
            return self.f.__get__(cls)
        return MethodType(self.f, cls)


class Property:
    "Emulate PyProperty_Type() in Objects/descrobject.c"

    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        if doc is None and fget is not None:
            doc = fget.__doc__
        self.__doc__ = doc

    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
        if self.fget is None:
            raise AttributeError("unreadable attribute")
        return self.fget(obj)

    def __set__(self, obj, value):
        if self.fset is None:
            raise AttributeError("can't set attribute")
        self.fset(obj, value)

    def __delete__(self, obj):
        if self.fdel is None:
            raise AttributeError("can't delete attribute")
        self.fdel(obj)

    def getter(self, fget):
        return type(self)(fget, self.fset, self.fdel, self.__doc__)

    def setter(self, fset):
        return type(self)(self.fget, fset, self.fdel, self.__doc__)

    def deleter(self, fdel):
        return type(self)(self.fget, self.fset, fdel, self.__doc__)

In [ ]:
class ClassProperty(property):
    "Emulate PyClassMethod_Type() in Objects/funcobject.c"

    def __init__(self, f, *args, **kwargs):
        f = property(f)
        super().__init__(f, *args, **kwargs)

    def __get__(self, obj, cls=None):
        if cls is None:
            cls = type(obj)
        if hasattr(type(self.fget), "__get__"):
            return self.fget.__get__(cls)
        return MethodType(self.fget, cls)

In [ ]:
A.__dict__["prop"].__dir__()

In [ ]:
A.myclsprop.__func__

In [ ]:
A.__dict__["myclsprop"].__dict__["__func__"]

In [ ]:
function = type(lambda: None)

In [ ]:
class _(function):
    ...

In [ ]:
A.__dict__["myclsprop"].__dir__()